# Floodwater Depth Estimation Tool (FwDET) version 2.1
Program procedure:
 1. Flood extent polygon to polyline
 2. Polyline to Raster - DEM extent and resolution (Env)
 3. Con - DEM values to Raster
 4. Euclidean Allocation - assign boundary cell elevation to nearest domain cells
 5. Calculate water depth by deducting DEM by Euclidean Allocation
 6. Run low-pass Filter
 
Created by Sagy Cohen, Surface Dynamics Modeling Lab, University of Alabama
email: sagy.cohen@ua.edu; web: http://sdml.ua.edu

## Set input and output variables 

In [1]:
# use kernel arcgispro-py3 (python 3.9.16)
import rasterio

In [2]:
#arcpy.env.overwriteOutput = True

infolder = 'fwDEToutput'
dem_path = 'C:/Users/Lyn/Documents/Programming/Python_codes/Impact_Map_Forecasting/Input_files/Houston_DEM_clip.tif'
inund_polygon = 'Fim_Beryl_5dayForecast.shp'


# make a geodatabase    
gdb_path = infolder 
# Create the geodatabase
#arcpy.CreateFileGDB_management(out_folder_path = infolder, out_name='geodatabase.gdb')
#workspace
#ws = arcpy.env.workspace = infolder


#clip_dem = 'Houston_DEM_clip.tif' #[Optional] - If empty, the clip_dem will be calculated with the Clip_management function
#cost_raster ='' #[Optional] - If empty, the CostRaster will be calculated below
WaterDepthOutput = 'WaterDepth_i10_s0p5'
iterations = 10
SlopeFiltering = True
SlopeThreshold = 0.5
#WaterDepthOutput = 'WaterDepth_i{0}_s{1}'.format(iterations,SlopeThreshold)

## Housekeeping

In [3]:

# Open the raster file using rasterio
with rasterio.open(dem_path) as dem:
    # Access cell height. Note: rasterio does not have a 'meanCellHeight' attribute directly.
    # Instead, you can use 'src.res' to get the cell width and height (resolution of the raster).
    cell_width, cell_height = dem.res
    crs = dem.crs
    # If you specifically need 'meanCellHeight', you can directly use 'cell_height',
    # assuming 'meanCellHeight' refers to the average or the height of each cell in the raster.
    meanCellHeight = cell_height


# Proper string representation of dem extent to be accepted by Clip_management method
#extent = '{} {} {} {}'.format(dem.extent.XMin, dem.extent.YMin, dem.extent.XMax, dem.extent.YMax)
extent = dem.bounds


# Print info about the raster
print(meanCellHeight)
print('coordinate system: ', crs)
print(extent)
    
## clip the dem to the extent if its too big ##
#clip_dem = 'ClipDEM'
#arcpy.management.Clip(dem, extent, clip_dem, inund_polygon, nodata_value= "-9999", clipping_geometry="ClippingGeometry", maintain_clipping_extent="NO_MAINTAIN_EXTENT")
#clip_dem_ras = arcpy.Raster(clip_dem)

9.391100523323995e-05
coordinate system:  EPSG:4326
BoundingBox(left=-96.124151208, bottom=28.92489112, right=-92.935677208, top=32.047338133)


In [6]:
# calculate cost raster

import rasterio
from rasterio.enums import Resampling
from rasterio.windows import Window

# Replace 'dem_name' with the path to your raster file
#dem_path = clip_dem  # Update this path

# Open the raster file using rasterio
with rasterio.open(dem_path) as src:
    # Copy the metadata from the source raster for use in the new raster
    meta = src.meta.copy()
    
    # Create a new raster file for the processed data
    with rasterio.open('CostRaster.tif', 'w', **meta) as dst:
        # Iterate over the raster in windows
        for ji, window in src.block_windows(1):
            # Read the data from the current window
            data = src.read(window=window)
            
            # Perform the operation: set values <= 0 to 999, then add 1 to all values
            cost_raster_data = (((data <= 0) * 999) + 1)
            
            # Write the modified data to the new raster, using the same window
            dst.write(cost_raster_data, window=window)
            



## Calculate boundary raster 

In [7]:
# convert polygon to polyline

from osgeo import ogr, gdal
import os

# Enable GDAL/OGR exceptions
gdal.UseExceptions()

source_file = inund_polygon
destination_file = infolder + '/polyline.shp'

# Open the source file
source_ds = gdal.OpenEx(source_file, gdal.OF_VECTOR)
if source_ds is None:
    raise RuntimeError(f"Unable to open {source_file}")

source_layer = source_ds.GetLayer()

# Create the destination file
driver = ogr.GetDriverByName('ESRI Shapefile')
if driver is None:
    raise RuntimeError("ESRI Shapefile driver not available")

# Delete the destination file if it already exists
if os.path.exists(destination_file):
    driver.DeleteDataSource(destination_file)

dest_ds = driver.CreateDataSource(destination_file)
if dest_ds is None:
    raise RuntimeError(f"Unable to create {destination_file}")

dest_layer = dest_ds.CreateLayer('line_layer', geom_type=ogr.wkbLineString)

# Iterate over the source layer polygons and convert them to lines
for feature in source_layer:
    geom = feature.GetGeometryRef()
    boundary = geom.Boundary()  # Get the boundary of the polygon, which is a line
    polyline = ogr.Feature(dest_layer.GetLayerDefn())
    polyline.SetGeometry(boundary)
    dest_layer.CreateFeature(polyline)
    polyline = None  # Destroy the feature to free resources

# Cleanup
source_ds = None
dest_ds = None

RuntimeError: Fim_Beryl_5dayForecast.shp: No such file or directory

In [ ]:
# convert polyline to raster
import geopandas as gpd
import rasterio
from rasterio.features import rasterize
from rasterio.transform import from_origin
import numpy as np

# Load the polyline shapefile using GeoPandas
polyline_gdf = gpd.read_file(polyline_path)

# Define the output raster parameters
cell_size = 10  # Example cell size, adjust as necessary
bounds = polyline_gdf.total_bounds
width = int((bounds[2] - bounds[0]) / cell_size)
height = int((bounds[3] - bounds[1]) / cell_size)

# Create a transform for the raster (assumes North up)
transform = from_origin(bounds[0], bounds[3], cell_size, cell_size)

# Rasterize the polyline
rasterized_polyline = rasterize(
    [(shape, 1) for shape in polyline_gdf.geometry],
    out_shape=(height, width),
    transform=transform,
    fill=0,  # Background value
    all_touched=True,  # Consider all pixels that the line touches
    dtype='uint8'
)

# Save the rasterized polyline to a new raster file
with rasterio.open(
    'rstr_poly.tif', 'w',
    driver='GTiff',
    height=height,
    width=width,
    count=1,
    dtype=rasterized_polyline.dtype,
    crs=polyline_gdf.crs,
    transform=transform,
) as dst:
    dst.write(rasterized_polyline, 1)

In [10]:
# convert polyline to raster
import geopandas as gpd
import rasterio
from rasterio.features import rasterize
from rasterio.transform import from_origin
import numpy as np
import math

# Load the polyline shapefile using GeoPandas
polyline_gdf = gpd.read_file(polyline_path)

if polyline_gdf.empty:
    raise ValueError("The GeoDataFrame is empty. Check your polyline_path.")

# Define the output raster parameters
cell_size = 10  # Example cell size, adjust as necessary
bounds = polyline_gdf.total_bounds
width = int((bounds[2] - bounds[0]) / cell_size)
height = int((bounds[3] - bounds[1]) / cell_size)

##  Process this in chunks ##

# Define chunk size (number of cells in each direction)
chunk_size_cells = 1000  # Adjust based on memory capacity

# Calculate number of chunks needed
num_chunks_x = math.ceil(width / chunk_size_cells)
num_chunks_y = math.ceil(height / chunk_size_cells)

# Function to rasterize a chunk
def rasterize_chunk(chunk_gdf, chunk_bounds, transform):
    width = int((chunk_bounds[2] - chunk_bounds[0]) / cell_size)
    height = int((chunk_bounds[3] - chunk_bounds[1]) / cell_size)
    return rasterize(
        [(shape, 1) for shape in chunk_gdf.geometry],
        out_shape=(height, width),
        transform=transform,
        fill=0,
        all_touched=True,
        dtype='uint8'
    )

# Process each chunk
for i in range(num_chunks_x):
    for j in range(num_chunks_y):
        # Calculate chunk bounds
        chunk_bounds = (
            bounds[0] + i * chunk_size_cells * cell_size,
            bounds[1] + j * chunk_size_cells * cell_size,
            bounds[0] + (i + 1) * chunk_size_cells * cell_size,
            bounds[1] + (j + 1) * chunk_size_cells * cell_size
        )
        
        # Clip polyline to chunk bounds
        chunk_gdf = polyline_gdf.cx[chunk_bounds[0]:chunk_bounds[2], chunk_bounds[1]:chunk_bounds[3]]
        
        if chunk_gdf.empty:
            continue  # Skip empty chunks
        
        # Create transform for the chunk
        chunk_transform = from_origin(chunk_bounds[0], chunk_bounds[3], cell_size, cell_size)
        
        # Rasterize the chunk
        rasterized_chunk = rasterize_chunk(chunk_gdf, chunk_bounds, chunk_transform)
        
        # Save or process the rasterized chunk...
        # This could involve saving each chunk as a separate file or combining them into one raster.
        # Implement saving or combining logic here.
        import numpy as np
        import rasterio
        from rasterio.transform import from_origin

        # Assuming variables like `total_width`, `total_height`, `cell_size`, and `bounds` are already defined

        # Initialize an empty array for the entire raster
        entire_raster = np.zeros((height, width), dtype='uint8')

        # Process each chunk (this code should be inside your loop for processing chunks)
        # Assuming `rasterized_chunk` is the rasterized data for the current chunk
        # and `i`, `j` are the current chunk indices in x and y directions respectively
        chunk_height, chunk_width = rasterized_chunk.shape
        start_x = i * chunk_size_cells
        start_y = j * chunk_size_cells
        end_x = start_x + chunk_width
        end_y = start_y + chunk_height
        window = Window(start_x, start_y, chunk_width, chunk_height)
        
        # Write the chunk's data to its position in the file
        dst.write(rasterized_chunk, window=window, indexes=1)

        import rasterio
        from rasterio.windows import Window

        # Initialize the raster file with full dimensions
        with rasterio.open(
            'combined_raster.tif', 'w',
            driver='GTiff',
            height=height,  # Total height of the raster
            width=width,  # Total width of the raster
            count=1,
            dtype='uint8',
            crs=polyline_gdf.crs,
            transform=transform,
        ) as dst:
            # Process each chunk (this part assumes you're inside a loop over chunks)
            for i in range(num_chunks_x):
                for j in range(num_chunks_y):
                    # Calculate the window position and size
                    start_x = i * chunk_size_cells
                    start_y = j * chunk_size_cells
                    window = Window(start_x, start_y, chunk_width, chunk_height)
            
                    # Write the chunk's data to its position in the file
                    dst.write(rasterized_chunk, window=window, indexes=1)

NameError: name 'polyline_path' is not defined

In [ ]:

import arcpy
#print('Focal iteration '+str(i+1))
arcpy.env.overwriteOutput = True
polyline = infolder + '/polyline.shp'
arcpy.PolygonToLine_management(inund_polygon, polyline)
# Convert polyline to raster
with arcpy.EnvManager(snapRaster=clip_dem_ras):
    arcpy.conversion.PolylineToRaster(polyline, 'FID', 'linerast15', "MAXIMUM_LENGTH", "NONE", cell_size)
raster_polyline = arcpy.Raster('linerast15')
raster_polyline.save("rstr_poly")
# The input whose values will be used as the output cell values if the condition is false.
inFalseConstant = '#'
where_clause = 'VALUE >= 0'
#Extract the boundary cells elevation from DEM
boundary = arcpy.sa.Con(raster_polyline, dem, inFalseConstant, where_clause)
# boundary.save('boundary1')
#Smooth boundary raster
for i in range(iterations):
    OutRasTemp = arcpy.sa.FocalStatistics(boundary, "Rectangle 5 5 CELL", 'MEAN', 'DATA')
    boundary = arcpy.sa.Con(raster_polyline, OutRasTemp, inFalseConstant, where_clause)
    boundary.save('boundary'+str(i+1))
#Identify and remove ocean boundary cells
OutRasTemp = arcpy.sa.FocalStatistics(dem, 'Circle 2 CELL', 'MINIMUM', 'DATA') 
whereClause2 = 'VALUE > 0'
boundary = arcpy.sa.Con(OutRasTemp, boundary, inFalseConstant, whereClause2)
#boundary.save("boundaryAfterOcean")
if SlopeFiltering:
#calculate topo slope
    print('Calculating Slope')
    extent_clip = '{} {} {} {}'.format(boundary.extent.XMin, boundary.extent.YMin, boundary.extent.XMax, boundary.extent.YMax)
    with arcpy.EnvManager(extent=extent_clip):
        out_slope = arcpy.sa.Slope(dem, "PERCENT_RISE", 1, "GEODESIC", "METER")
        out_slope.save("Slope_m")
#Remove erroneous boundary cells 
    whereClause_slope = 'VALUE > ' + str(SlopeThreshold)
    boundary = arcpy.sa.Con(out_slope, boundary, inFalseConstant, whereClause_slope)
boundary.save("boundFinal")
print('Finished calculating the boundary raster')

## Calculate the water depth raster

In [ ]:
MULTIPLIER = 10000
boundary_int = arcpy.sa.Int(boundary * MULTIPLIER)
#boundary_int.save("boundary_int")
print('Running cost allocation')
with arcpy.EnvManager(snapRaster=None, extent="DEFAULT", mask=clip_dem):
    cost_alloc = arcpy.sa.CostAllocation(boundary_int, cost_raster, '#', '#', 'Value')

# Divide the result from the cost allocation function using the same constant used to create the integer
# representation of the boundary
cost_alloc = arcpy.sa.Float(cost_alloc) / MULTIPLIER
print('Cost Allocation raster generated')
print('Calculating estimated water depth')
water_depth = (cost_alloc - clip_dem_ras)
# Remove estimated water depths below 0 and change them to 0
water_depth = arcpy.sa.Con(water_depth > 0, water_depth,"#")
#water_depth.save(WaterDepthOutput)
#print('Floodwater depth computed')
#water_depth

## Calculate smooth water depth raster (low-pass filter)

In [ ]:
#Run a low-pass filter
print('Running low-pass Filter')
water_depth_filtered = arcpy.sa.Filter(water_depth, 'LOW', 'DATA')
waterDepthFilter2 = arcpy.sa.Con(clip_dem_ras, water_depth_filtered, '#', 'VALUE > 0')
#waterDepthFilter2.save(WaterDepthOutput+'_filtered')
print('Finished low-pass Filter calculation')
waterDepthFilter2

In [ ]:
%pip install rasterio

In [ ]:
# Save the waterDepthFilter2 raster to a file
waterDepthFilter2.save(infolder+"waterdepth.tif")